<a href="https://colab.research.google.com/github/quinn-dougherty/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/module1-logistic-regression/logis_regr_heartdisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
dat_url = 'https://web.stanford.edu/~hastie/ElemStatLearn/datasets/SAheart.data'

'''A retrospective sample of males in a heart-disease high-risk region
of the Western Cape, South Africa. There are roughly two controls per
case of CHD. Many of the CHD positive men have undergone blood
pressure reduction treatment and other programs to reduce their risk
factors after their CHD event. In some cases the measurements were
made after these treatments. These data are taken from a larger
dataset, described in  Rousseauw et al, 1983, South African Medical
Journal. 

sbp		systolic blood pressure
tobacco		cumulative tobacco (kg)
ldl		low densiity lipoprotein cholesterol
adiposity
famhist		family history of heart disease (Present, Absent)
typea		type-A behavior
obesity
alcohol		current alcohol consumption
age		age at onset
chd		response, coronary heart disease
'''

import pandas as pd
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from numpy.testing import assert_almost_equal
from functools import reduce

In [72]:
df = pd.read_csv(dat_url).drop('row.names', axis=1)

assert all(map(lambda x: x==0, list(df.isna().sum().values)))

df.head()


,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


In [73]:
df.describe()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd
count,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000
mean,138.326840,3.635649,4.740325,25.406732,53.103896,26.044113,17.044394,42.816017,0.346320
std,20.496317,4.593024,2.070909,7.780699,9.817534,4.213680,24.481059,14.608956,0.476313
min,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000,0.000000
25%,124.000000,0.052500,3.282500,19.775000,47.000000,22.985000,0.510000,31.000000,0.000000
50%,134.000000,2.000000,4.340000,26.115000,53.000000,25.805000,7.510000,45.000000,0.000000
75%,148.000000,5.500000,5.790000,31.227500,60.000000,28.497500,23.892500,55.000000,1.000000
max,218.000000,31.200000,15.330000,42.490000,78.000000,46.580000,147.190000,64.000000,1.000000


In [102]:
X = df.drop(['chd', 'famhist'], axis=1)
X['famhist'] = df.famhist.map({'Present': 1, 'Absent': 0})
y = df.chd

sclr = StandardScaler()
sclr.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,y)

#logi_mod.classes_, logi_mod.coef_, logi_mod.intercept_

#help(logi_mod)

#logi_mod.predict(X)

''' https://stackoverflow.com/questions/52670012/convergencewarning-liblinear-failed-to-converge-increase-the-number-of-iterati

Normally when an optimization algorithm does not converge, it is usually because the problem is not well-conditioned, perhaps due to a poor scaling of the decision variables. There are a few things you can try.

    Normalize your training data so that the problem hopefully becomes more well conditioned, which in turn can speed up convergence. One possibility is to scale your data to 0 mean, unit standard deviation using Scikit-Learn's StandardScaler for an example. Note that you have to apply the StandardScaler fitted on the training data to the test data.
    Related to 1), make sure the other arguments such as regularization weight, C, is set appropriately.
    Set max_iter to a larger value. The default is 1000.

'''

logi_mod = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [108]:
#logi_mod.classes_, logi_mod.coef_, logi_mod.intercept_

assert_almost_equal(1 - np.divide(abs(logi_mod.predict(X_test) - y_test).sum(), X_test.shape[0]), 
                    np.divide(sum(logi_mod.predict(X_test) == y_test), X_test.shape[0])
                   )

print('error: {}'.format(np.divide(sum(logi_mod.predict(X_test) == y_test), X_test.shape[0])))
print('R2 (variance explained): {}'.format(logi_mod.score(X_test, y_test)))

np.sqrt(logi_mod.score(X_test, y_test)) # pearson's correlation coefficient 

error: 0.6896551724137931
R2 (variance explained): 0.6896551724137931


0.8304547985373997